In [1]:
!pip install datapackage
!pip install geopandas

     |████████████████████████████████| 85 kB 2.6 MB/s 
     |████████████████████████████████| 68 kB 6.0 MB/s 
     |████████████████████████████████| 72 kB 275 kB/s 
     |████████████████████████████████| 45 kB 2.7 MB/s 
     |████████████████████████████████| 131 kB 54.5 MB/s 
     |████████████████████████████████| 242 kB 56.3 MB/s 
     |████████████████████████████████| 126 kB 72.5 MB/s 
     |████████████████████████████████| 8.2 MB 41.8 MB/s 
     |████████████████████████████████| 79 kB 8.1 MB/s 
     |████████████████████████████████| 127 kB 68.4 MB/s 
  Created wheel for linear-tsv: filename=linear_tsv-1.1.0-py3-none-any.whl size=7399 sha256=6d6e07eb93634d1129fd22f6fa3f7f979bf9a541ef14c38f8838cb4661e9fdaf
  Stored in directory: /root/.cache/pip/wheels/dd/0a/62/49ee84fab92057fb3e303fb1a59e75ccac6899b417f441305a
  Created wheel for unicodecsv: filename=unicodecsv-0.14.1-py3-none-any.whl size=10765 sha256=f592ec1ef5f7fc9453f207ad88d6efb21cbe0b9f034ba76bebc9bf6ac712a697
  Store

In [2]:
import os, sys, subprocess
import numpy                 as np
import datapackage
import geopandas
import pandas                as pd
import matplotlib.pyplot     as plt
import seaborn               as sns
import folium  #needed for interactive map
from folium.plugins import HeatMap
from ipywidgets import interact, SelectionRangeSlider
from datetime import datetime
from matplotlib import collections  as mc

from sklearn import linear_model
from scipy import stats

%matplotlib inline
sns.set()

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
main = pd.read_csv('/content/drive/MyDrive/DS4A-W_Team25/Colab_Files/data/global_data/master_df.csv')
main.head(3)

In [ ]:
# Load the geopandas world dataset
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
print(world.head(3))
world.plot()

In [ ]:
# rename 'iso' in 'main' to 'iso_a3' in order to merge with the world dataset
main = main.rename(columns={"iso": "iso_a3"})
combined = world.merge(main)
combined.head(3)

In [ ]:
# What dates do I have data for 'number_of_disasters'?
temp = main[['year','number_of_disasters']]
temp = temp.dropna()
print(temp['year'].min())
print(temp['year'].max())

daterange = np.linspace(1900,2021,122)
daterange = [int(x) for x in daterange]

In [ ]:
# Create function to run to plot world map of number of disasters in interactive map
def disasters_slider(date):
    fig,ax = plt.subplots(figsize = (12,5), facecolor = plt.cm.Blues(.2))    
    ax.set_facecolor(plt.cm.Blues(.2))
    world.plot(
        ax = ax,
        color = plt.cm.Blues(.2),
        edgecolor = 'white'
    )
    combined[combined['year'] == date].plot(ax=ax,column='number_of_disasters', legend = True)


In [ ]:
# interactive widget time!!!
interact(disasters_slider,date=(1900,2021,1))

In [ ]:
# What dates do I have data for 'rice_production_in_tonnes'?
temp = main[['year','rice_production_in_tonnes']]
temp = temp.dropna()
print(temp['year'].min())
print(temp['year'].max())

daterange = np.linspace(1900,2021,122)
daterange = [int(x) for x in daterange]

In [ ]:
# Create function to run to plot world map of rice production in interactive map
def rice_production_slider(date):
    fig,ax = plt.subplots(figsize = (12,5), facecolor = plt.cm.Blues(.2))    
    ax.set_facecolor(plt.cm.Blues(.2))
    world.plot(
        ax = ax,
        color = plt.cm.Blues(.2),
        edgecolor = 'white'
    )
    combined[combined['year'] == date].plot(ax=ax,column='rice_production_in_tonnes', legend = True)


In [ ]:
# interactive widget time!!!
interact(rice_production_slider,date=(1961,2019,1))

## Scatterplot precipitation vs rice production

In [ ]:
# Group data for precipitation, rice production, year, and country
df = main[['year','iso_a3','precipitation','temperature','rice_production_in_tonnes']]
df = df.dropna()
df.head(3)

In [ ]:
fig, ax = plt.subplots()

for i in df[df.iso_a3 == 'CHN']:
    sns.scatterplot(x='precipitation',y='rice_production_in_tonnes', data=df[df.iso_a3 == i])

In [ ]:
df_chn = df[df.iso_a3 == 'CHN'].sort_values(by=['year'])

N = df_chn['year'].max() - df_chn['year'].min()

colors = [plt.cm.viridis(x) for x in np.linspace(0,1,N+1)]

fig,ax = plt.subplots()

x = df_chn.rice_production_in_tonnes
y = df_chn.precipitation
years = df_chn.year.unique()

ax.plot(x,y)
cset1 = ax.scatter(x,y, color=colors)
ax.set_ylabel('Precipitation')
ax.set_xlabel('Rice Production (tonnes)')


In [ ]:
df_chn = df[df.iso_a3 == 'CHN'].sort_values(by=['year'])

N = df_chn['year'].max() - df_chn['year'].min()

colors = [plt.cm.viridis(x) for x in np.linspace(0,1,N+1)]

fig,ax = plt.subplots()

x = df_chn.rice_production_in_tonnes
y = df_chn.temperature
years = df_chn.year.unique()

ax.plot(x,y)
cset1 = ax.scatter(x,y, color=colors)
ax.set_ylabel('temperature')
ax.set_xlabel('Rice Production (tonnes)')


## Correlation Heatmap for different years in CHINA

In [ ]:
df_chn = main[main.iso_a3=='CHN']
df_chn = df_chn.dropna()
df_chn.head(10)

In [ ]:
df_chn.columns

In [ ]:
corr = df_chn[['number_of_disasters',
       'total_human_affected', 'temperature', 'precipitation', 'cropland',
       'country_area', 'cropland_pct', 'population',
       'rice_production_in_tonnes']].corr()

fig,ax = plt.subplots(figsize = (8,8))
sns.heatmap(corr, center=0,  annot=True, ax=ax)

In [ ]:
# add rice_production_per_capita to graph and do heatmap again, without population
main['rice_per_capita'] = main['rice_production_in_tonnes'] / main['population']
main.head(3)

In [ ]:
def corrmatrix(ISO):

    df_iso = main[main.iso_a3==ISO]
    df_iso = df_iso.dropna()

    corr = df_iso[['number_of_disasters',
           'total_human_affected', 'temperature', 'precipitation', 'cropland',
           'country_area', 'cropland_pct',
           'rice_per_capita']].corr()

    fig,ax = plt.subplots(figsize = (8,8))
    sns.heatmap(corr, center=0,  annot=True, ax=ax)


In [ ]:
corrmatrix('CHN')
corrmatrix('USA')
corrmatrix('IND')
corrmatrix('USA')
corrmatrix('JPN')

## Who are the main rice producers?

In [ ]:
producers = main[['iso_a3','year','country_or_area','rice_production_in_tonnes']]
producers = producers.dropna()
producers.head(10)

In [ ]:
# In this case, 'China' is the sum of all Chinese territories. Remove it and replace the country code for China, mainland, with CHN.

# column country_or_area has value 'China'
index_names = producers[ producers['country_or_area'] == 'China' ].index
  
# drop these row indexes
# from dataFrame
producers.drop(index_names, inplace = True)

# replace iso value for 'China, mainland' with 'CHN'
producers["iso_a3"].loc[producers.country_or_area == 'China, mainland'] = 'CHN'

In [ ]:
# Find top 10 producers in each year
topproducers = pd.DataFrame(columns=producers.columns)

for yr in producers['year'].unique():
    temp = producers[producers.year == yr].sort_values(by=['rice_production_in_tonnes'], ascending=False)
    topproducers = topproducers.append(temp.head(10))

topproducers

In [ ]:
topproducers[(topproducers.iso_a3 == 'CHN') & (topproducers.year == 1980)]['rice_production_in_tonnes'].any()

In [ ]:
countrylist = topproducers.iso_a3.unique()
years = topproducers.year.unique()

fig, ax = plt.subplots()

colors = [plt.cm.viridis(x) for x in np.linspace(0,1,len(countrylist)+1)]

m=0
for i in countrylist:
    for j in years:
        if topproducers[(topproducers.iso_a3 == i) & (topproducers.year == j)]['rice_production_in_tonnes'].any() == True:
            ax.scatter(j,i, marker='s', color=colors[m])
    m += 1

ax.set_yticks(ticks = np.arange(0, len(countrylist)))
ax.set_yticklabels(countrylist)
ax.set_ylabel('top 10 rice producing countries')
ax.set_xlabel('years')

## Create Linear Regression

In [ ]:
def linregplot(main_nona):
    
    """ Feed in dataset, cleaned from 'main', generate regression plot """
    
    # Define independent variable (rice production in China)
    y = main_nona['rice_production_in_tonnes']

    # Define dependent variables
    X = main_nona[['number_of_disasters', 'temperature', 'precipitation','cropland_pct']]

    regr = linear_model.LinearRegression()
    regr.fit(X, y)

    predictions = regr.predict(X)

    # generate y = mx+b line
    slope, intercept, r, p, std_err = stats.linregress(y, predictions)
    def myfunc(x):
        return slope * x + intercept

    linearpred = myfunc(y)

    # Create dataframe with output data
    ydata = main_nona[['year','iso_a3','country_or_area','rice_production_in_tonnes']].copy(deep=True)
    ydata['predicted'] = predictions
    ydata['linear regression'] = linearpred
    
    return ydata

### First drop all na values and drop all zero values and plot regression

In [ ]:
# drop all na values
temp = main.dropna()
# drop rows if 'rice_production_in_tonnes' is zero
index_names = temp[temp['rice_production_in_tonnes'] == 0 ].index
temp.drop(index_names, inplace = True)

ydata = linregplot(temp)

In [ ]:
ydata.head(10)

In [ ]:
# Plot true values versus predicted values and linear regression, for each country
fig,ax = plt.subplots()
sns.scatterplot(ax=ax, x='rice_production_in_tonnes',y='predicted',data=ydata,hue='iso_a3')
sns.lineplot(ax=ax, x='rice_production_in_tonnes',y='linear regression',data=ydata)
ax.get_legend().remove()

### Filter for only rice production > 0.15e8

In [ ]:
# drop all na values
temp = main.dropna()
# drop rows if 'rice_production_in_tonnes' is zero
index_names = temp[temp['rice_production_in_tonnes'] < 0.15e8 ].index
temp.drop(index_names, inplace = True)

ydata = linregplot(temp)

# Plot true values versus predicted values and linear regression, for each country
fig,ax = plt.subplots()
sns.scatterplot(ax=ax, x='rice_production_in_tonnes',y='predicted',data=ydata,hue='country_or_area')
sns.lineplot(ax=ax, x='rice_production_in_tonnes', y='linear regression',data=ydata)
ax.legend(bbox_to_anchor=(1.1, 1.05))

In [ ]:
# Correlation between population and cropland_pct
fig, ax = plt.subplots()

sns.scatterplot(x='population', y='cropland_pct',data=main, hue='iso_a3')
ax.get_legend().remove()